In [ ]:
import os
import glob
import random
import numpy as np
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.distributed as dist
import torch.multiprocessing as mp
import nibabel as nib
from torch.utils.data import Dataset, DataLoader
from torch.nn import CrossEntropyLoss
from torch.optim import AdamW
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts
from torch.cuda.amp import autocast, GradScaler
from sklearn.model_selection import StratifiedKFold
from skimage.filters import threshold_otsu
from scipy import ndimage
from tqdm import tqdm
import json
from datetime import datetime

# ---------------------------------------
# CUDA Ayarları
# ---------------------------------------
def setup_gpu():
    if torch.cuda.device_count() > 1:
        print(f"Using {torch.cuda.device_count()} GPUs!")
        # Set default device to cuda:0
        torch.cuda.set_device(0)
        # Her GPU için bellek optimizasyonu
        for i in range(torch.cuda.device_count()):
            torch.cuda.empty_cache()
    else:
        print("Using single GPU or CPU")
    
    torch.backends.cudnn.benchmark = True
    torch.backends.cudnn.deterministic = True
    return torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

DEVICE = setup_gpu()

# ---------------------------------------
# 1) Veri Klasörü ve Sınıfların Otomatik Tespiti
# ---------------------------------------
DATA_DIR = '/kaggle/input/skullstripping'
all_classes = [d for d in os.listdir(DATA_DIR) if os.path.isdir(os.path.join(DATA_DIR, d))]
classes = {name: idx for idx, name in enumerate(sorted(all_classes))}
num_classes = len(classes)
print(f"Detected classes: {classes}")

# ---------------------------------------
# 2) ".nii" Dosyalarını Listeleyip Etiketleme
# ---------------------------------------
all_nii = glob.glob(os.path.join(DATA_DIR, '**', '*.nii'), recursive=True)
paths, labels = [], []
for p in all_nii:
    parent = os.path.dirname(p)
    grand = os.path.dirname(parent)
    cls_name = os.path.basename(grand)
    if cls_name not in classes:
        continue
    if os.path.getsize(p) == 0:
        continue
    paths.append(p)
    labels.append(classes[cls_name])

if len(set(labels)) < 2:
    raise RuntimeError(f"En az iki farklı sınıf bulunmalı; mevcut: {set(labels)}")

for cls, idx in classes.items():
    print(f"  {cls}: {labels.count(idx)} örnek")

# Shuffle
tuple_data = list(zip(paths, labels))
random.seed(42)
random.shuffle(tuple_data)
paths, labels = zip(*tuple_data)
paths, labels = list(paths), list(labels)

# ---------------------------------------
# 3) Dataset ve Data Augmentations
# ---------------------------------------
PATCH_SIZE = 128  # Increased patch size for better feature extraction

def random_flip(tensor):
    for axis in [1, 2, 3]:
        if random.random() < 0.5:
            tensor = tensor.flip(axis)
    return tensor

def random_rotate(tensor):
    if random.random() < 0.5:
        angle = random.choice([90, 180, 270])
        tensor = torch.rot90(tensor, k=angle//90, dims=[2, 3])
    return tensor

def random_noise(tensor):
    if random.random() < 0.3:
        noise = torch.randn_like(tensor) * 0.1
        tensor = tensor + noise
    return tensor

class SkullStripDataset(Dataset):
    def __init__(self, paths, labels, augment=False):
        self.paths = paths
        self.labels = labels
        self.augment = augment

    def __len__(self):
        return len(self.paths)

    def __getitem__(self, idx):
        img = nib.load(self.paths[idx]).get_fdata().astype(np.float32)
        
        # Enhanced preprocessing
        p1, p99 = np.percentile(img, (1, 99))
        img = np.clip(img, p1, p99)
        img = (img - p1) / (p99 - p1 + 1e-6)
        
        # Improved skull stripping
        mask = img > threshold_otsu(img)
        mask = ndimage.binary_closing(mask, structure=np.ones((3,3,3)))
        mask = ndimage.binary_fill_holes(mask)
        labeled, num = ndimage.label(mask)
        if num > 1:
            counts = np.bincount(labeled.ravel()); counts[0] = 0
            mask = (labeled == counts.argmax())
        
        # Enhanced patch extraction
        coords = np.array(np.where(mask))
        mins = coords.min(axis=1); maxs = coords.max(axis=1)
        centers = (mins + maxs) // 2
        half = PATCH_SIZE // 2
        starts = centers - half; ends = starts + PATCH_SIZE
        
        # Boundary handling
        for i in range(3):
            d = img.shape[i]
            if starts[i] < 0:
                starts[i] = 0; ends[i] = PATCH_SIZE
            if ends[i] > d:
                ends[i] = d; starts[i] = d - PATCH_SIZE
        
        patch = img[starts[0]:ends[0], starts[1]:ends[1], starts[2]:ends[2]]
        
        # Padding if needed
        if patch.shape != (PATCH_SIZE,)*3:
            pad = np.zeros((PATCH_SIZE,)*3, dtype=np.float32)
            pd, ph, pw = patch.shape
            sd, sh, sw = [(PATCH_SIZE - s)//2 for s in patch.shape]
            pad[sd:sd+pd, sh:sh+ph, sw:sw+pw] = patch
            patch = pad
        
        # Enhanced normalization
        patch = (patch - patch.mean()) / (patch.std() + 1e-6)
        tensor = torch.from_numpy(patch).unsqueeze(0)
        
        if self.augment:
            tensor = random_flip(tensor)
            tensor = random_rotate(tensor)
            tensor = random_noise(tensor)
            
        return tensor, torch.tensor(self.labels[idx], dtype=torch.long)

# ---------------------------------------
# 4) Enhanced SE Block and Inception3D_SE Model
# ---------------------------------------
class SEBlock(nn.Module):
    def __init__(self, channels, reduction=8):
        super().__init__()
        self.fc1 = nn.Linear(channels, channels//reduction, bias=False)
        self.relu = nn.ReLU(inplace=True)
        self.fc2 = nn.Linear(channels//reduction, channels, bias=False)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x):
        b,c,d,h,w = x.size()
        y = x.view(b,c,-1).mean(dim=2)
        y = self.relu(self.fc1(y))
        y = self.sigmoid(self.fc2(y)).view(b,c,1,1,1)
        return x * y

class Inception3D_SE(nn.Module):
    def __init__(self, in_ch, c1, r3, c3, r5, c5, p):
        super().__init__()
        self.b1 = nn.Sequential(
            nn.Conv3d(in_ch, c1, kernel_size=1, bias=False),
            nn.BatchNorm3d(c1, affine=True),
            nn.ReLU(inplace=True)
        )
        self.b3 = nn.Sequential(
            nn.Conv3d(in_ch, r3, kernel_size=1, bias=False),
            nn.BatchNorm3d(r3, affine=True),
            nn.ReLU(inplace=True),
            nn.Conv3d(r3, c3, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm3d(c3, affine=True),
            nn.ReLU(inplace=True)
        )
        self.b5 = nn.Sequential(
            nn.Conv3d(in_ch, r5, kernel_size=1, bias=False),
            nn.BatchNorm3d(r5, affine=True),
            nn.ReLU(inplace=True),
            nn.Conv3d(r5, c5, kernel_size=5, padding=2, bias=False),
            nn.BatchNorm3d(c5, affine=True),
            nn.ReLU(inplace=True)
        )
        self.bp = nn.Sequential(
            nn.MaxPool3d(kernel_size=3, stride=1, padding=1),
            nn.Conv3d(in_ch, p, kernel_size=1, bias=False),
            nn.BatchNorm3d(p, affine=True),
            nn.ReLU(inplace=True)
        )
        out = c1 + c3 + c5 + p
        self.se = SEBlock(out)
        
    def forward(self, x):
        return self.se(torch.cat([
            self.b1(x), self.b3(x), self.b5(x), self.bp(x)
        ], dim=1))

class Inception3DNet_SE(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.conv1 = nn.Sequential(
            nn.Conv3d(1, 64, kernel_size=7, stride=2, padding=3, bias=False),
            nn.BatchNorm3d(64, affine=True),
            nn.ReLU(inplace=True),
            nn.MaxPool3d(kernel_size=3, stride=2, padding=1)
        )
        self.conv2 = nn.Sequential(
            nn.Conv3d(64, 64, kernel_size=1, bias=False),
            nn.BatchNorm3d(64, affine=True),
            nn.ReLU(inplace=True),
            nn.Conv3d(64, 192, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm3d(192, affine=True),
            nn.ReLU(inplace=True),
            nn.MaxPool3d(kernel_size=3, stride=2, padding=1)
        )
        self.i3a = Inception3D_SE(192, 64, 96, 128, 16, 32, 32)
        self.i3b = Inception3D_SE(256, 128, 128, 192, 32, 96, 64)
        self.mp3 = nn.MaxPool3d(kernel_size=3, stride=2, padding=1)
        self.i4a = Inception3D_SE(480, 192, 96, 208, 16, 48, 64)
        self.i4b = Inception3D_SE(512, 160, 112, 224, 24, 64, 64)
        self.i4c = Inception3D_SE(512, 128, 128, 256, 24, 64, 64)
        self.ap = nn.AdaptiveAvgPool3d((1, 1, 1))
        self.do = nn.Dropout(0.4)
        self.fc = nn.Linear(512, num_classes)
        
        # Initialize weights
        for m in self.modules():
            if isinstance(m, nn.Conv3d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, (nn.BatchNorm3d, nn.GroupNorm)):
                if hasattr(m, 'weight') and m.weight is not None:
                    nn.init.constant_(m.weight, 1)
                if hasattr(m, 'bias') and m.bias is not None:
                    nn.init.constant_(m.bias, 0)
                    
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.i3a(x)
        x = self.i3b(x)
        x = self.mp3(x)
        x = self.i4a(x)
        x = self.i4b(x)
        x = self.i4c(x)
        x = self.ap(x).view(x.size(0), -1)
        x = self.do(x)
        return self.fc(x)

# ---------------------------------------
# Hiperparametre Yönetimi
# ---------------------------------------
class HyperParameterManager:
    def __init__(self):
        self.base_params = {
            'learning_rate': 2e-4,
            'weight_decay': 1e-5,
            'dropout_rate': 0.4,
            'batch_size': 16 if DEVICE.type == 'cuda' else 4,  # 2xT4 için artırıldı
            'patch_size': 128,
            'se_reduction': 8,
            'num_workers': 4,  # Her GPU için worker sayısı
            'gradient_accumulation_steps': 2  # Gradient biriktirme adımları
        }
        self.fold_results = []
        self.best_params = self.base_params.copy()
        self.best_accuracy = 0.0
        
    def update_params(self, fold_results):
        """Fold sonuçlarına göre hiperparametreleri güncelle"""
        self.fold_results.append(fold_results)
        
        if len(self.fold_results) >= 3:
            recent_results = self.fold_results[-3:]
            avg_accuracy = np.mean([r['val_accuracy'] for r in recent_results])
            avg_loss = np.mean([r['val_loss'] for r in recent_results])
            
            if avg_accuracy < 0.7:
                self.best_params['learning_rate'] *= 0.8
                self.best_params['weight_decay'] *= 1.2
                self.best_params['dropout_rate'] = min(0.5, self.best_params['dropout_rate'] * 1.1)
                
            if avg_loss > 1.0:
                self.best_params['batch_size'] = max(8, self.best_params['batch_size'] - 4)
                self.best_params['patch_size'] = min(160, self.best_params['patch_size'] + 8)
                
            if avg_accuracy > self.best_accuracy:
                self.best_accuracy = avg_accuracy
                self.best_params = self.base_params.copy()
                
        return self.best_params
    
    def save_results(self, fold_num):
        """Sonuçları JSON dosyasına kaydet"""
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        results = {
            'timestamp': timestamp,
            'fold_results': self.fold_results,
            'best_params': self.best_params,
            'best_accuracy': self.best_accuracy
        }
        
        with open(f'training_results_fold{fold_num}_{timestamp}.json', 'w') as f:
            json.dump(results, f, indent=4)

# ---------------------------------------
# 5) Eğitim Döngüsü
# ---------------------------------------
NUM_FOLDS = 5
EPOCHS = 10

def train_fold(fold, train_idx, val_idx, hp_manager):
    print(f"\n>>> Fold {fold}/{NUM_FOLDS}")
    
    current_params = hp_manager.best_params
    
    # Dataset'leri oluştur
    train_ds = SkullStripDataset([paths[i] for i in train_idx], 
                                [labels[i] for i in train_idx], 
                                augment=True)
    val_ds = SkullStripDataset([paths[i] for i in val_idx], 
                              [labels[i] for i in val_idx], 
                              augment=False)

    # DataLoader'ları oluştur
    train_loader = DataLoader(train_ds, 
                            batch_size=current_params['batch_size'], 
                            shuffle=True,
                            num_workers=current_params['num_workers'],
                            pin_memory=True,
                            persistent_workers=True)
    val_loader = DataLoader(val_ds, 
                          batch_size=current_params['batch_size'], 
                          shuffle=False,
                          num_workers=current_params['num_workers'],
                          pin_memory=True,
                          persistent_workers=True)

    # Model oluştur ve çoklu GPU'ya dağıt
    model = Inception3DNet_SE(num_classes)
    model = model.to(DEVICE)  # First move to default device (cuda:0)
    if torch.cuda.device_count() > 1:
        model = nn.DataParallel(model)
    
    # Fix dropout rate access for DataParallel
    if isinstance(model, nn.DataParallel):
        model.module.do.p = current_params['dropout_rate']
    else:
        model.do.p = current_params['dropout_rate']
    
    criterion = CrossEntropyLoss(weight=class_weights.to(DEVICE))  # Move weights to default device
    optimizer = AdamW(model.parameters(), 
                     lr=current_params['learning_rate'], 
                     weight_decay=current_params['weight_decay'])
    scheduler = CosineAnnealingWarmRestarts(optimizer, T_0=5, T_mult=2, eta_min=1e-6)
    
    scaler = GradScaler()
    best_acc = 0
    no_improve = 0
    patience = 5
    
    fold_metrics = {
        'train_losses': [],
        'train_accuracies': [],
        'val_losses': [],
        'val_accuracies': [],
        'learning_rates': []
    }

    for epoch in range(1, EPOCHS+1):
        print(f"Epoch {epoch}/{EPOCHS}")
        model.train()
        tloss = 0
        corr = 0
        tot = 0
        
        # Gradient biriktirme için
        optimizer.zero_grad(set_to_none=True)
        
        for batch_idx, (imgs, lbls) in enumerate(tqdm(train_loader, desc="Train")):
            imgs, lbls = imgs.to(DEVICE), lbls.to(DEVICE)
            
            with autocast():
                logits = model(imgs)
                loss = criterion(logits, lbls)
                # Gradient biriktirme için loss'u normalize et
                loss = loss / current_params['gradient_accumulation_steps']

            scaler.scale(loss).backward()
            
            if (batch_idx + 1) % current_params['gradient_accumulation_steps'] == 0:
                scaler.unscale_(optimizer)
                nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
                scaler.step(optimizer)
                scaler.update()
                optimizer.zero_grad(set_to_none=True)
            
            tloss += loss.item() * current_params['gradient_accumulation_steps']
            preds = logits.argmax(1)
            corr += (preds==lbls).sum().item()
            tot += lbls.size(0)
        
        train_loss = tloss/len(train_loader)
        train_acc = corr/tot
        print(f"  Train Loss: {train_loss:.4f}, Acc: {train_acc:.4f}")
        
        # Validation
        model.eval()
        vloss = 0
        corr = 0
        tot = 0
        with torch.no_grad():
            for imgs, lbls in tqdm(val_loader, desc="Val"):
                imgs, lbls = imgs.to(DEVICE), lbls.to(DEVICE)
                with autocast():
                    logits = model(imgs)
                    loss = criterion(logits, lbls)

                vloss += loss.item()
                preds = logits.argmax(1)
                corr += (preds==lbls).sum().item()
                tot += lbls.size(0)
        
        val_loss = vloss/len(val_loader)
        val_acc = corr/tot
        print(f"  Val Loss: {val_loss:.4f}, Acc: {val_acc:.4f}")
        
        fold_metrics['train_losses'].append(train_loss)
        fold_metrics['train_accuracies'].append(train_acc)
        fold_metrics['val_losses'].append(val_loss)
        fold_metrics['val_accuracies'].append(val_acc)
        fold_metrics['learning_rates'].append(optimizer.param_groups[0]['lr'])
        
        scheduler.step()
        
        if val_acc > best_acc + 1e-4:
            best_acc = val_acc
            # Model checkpoint'ini kaydet
            if isinstance(model, nn.DataParallel):
                model_state = model.module.state_dict()
            else:
                model_state = model.state_dict()
                
            torch.save({
                'epoch': epoch,
                'model_state_dict': model_state,
                'optimizer_state_dict': optimizer.state_dict(),
                'scheduler_state_dict': scheduler.state_dict(),
                'best_acc': best_acc,
                'hyperparameters': current_params
            }, f"best_fold{fold}.pth")
            print("  + New best! Saved.")
            no_improve = 0
        else:
            no_improve += 1
        if no_improve >= patience:
            print("  Early stopping")
            break
    
    return {
        'fold': fold,
        'best_accuracy': best_acc,
        'final_val_loss': val_loss,
        'final_val_accuracy': val_acc,
        'epochs_trained': epoch,
        'hyperparameters': current_params,
        'metrics': fold_metrics
    }

# Ana eğitim döngüsü
if __name__ == '__main__':
    # Hiperparametre yöneticisini başlat
    hp_manager = HyperParameterManager()
    
    # Calculate class weights
    class_counts = np.bincount(labels)
    total_samples = len(labels)
    class_weights = torch.FloatTensor(total_samples / (len(class_counts) * class_counts)).to(DEVICE)
    print(f"Class weights: {class_weights}")
    
    # K-fold cross validation
    skf = StratifiedKFold(n_splits=NUM_FOLDS, shuffle=True, random_state=42)
    all_idx = np.arange(len(paths))
    labels_arr = np.array(labels)
    
    all_fold_results = []
    
    for fold, (train_idx, val_idx) in enumerate(skf.split(all_idx, labels_arr), 1):
        fold_result = train_fold(fold, train_idx, val_idx, hp_manager)
        all_fold_results.append(fold_result)
        
        # Hiperparametreleri güncelle
        hp_manager.update_params(fold_result)
        
        # Sonuçları kaydet
        hp_manager.save_results(fold)
        
        print(f"\nFold {fold} Summary:")
        print(f"Best Accuracy: {fold_result['best_accuracy']:.4f}")
        print(f"Final Validation Loss: {fold_result['final_val_loss']:.4f}")
        print(f"Final Validation Accuracy: {fold_result['final_val_accuracy']:.4f}")
        print(f"Epochs Trained: {fold_result['epochs_trained']}")
        print("\nCurrent Hyperparameters:")
        for param, value in fold_result['hyperparameters'].items():
            print(f"  {param}: {value}")
    
    # Final sonuçları
    print("\n=== Final Results ===")
    avg_accuracy = np.mean([r['best_accuracy'] for r in all_fold_results])
    std_accuracy = np.std([r['best_accuracy'] for r in all_fold_results])
    print(f"Average Accuracy: {avg_accuracy:.4f} ± {std_accuracy:.4f}")
    print("\nBest Hyperparameters:")
    for param, value in hp_manager.best_params.items():
        print(f"  {param}: {value}")
    
    # Tüm sonuçları kaydet
    final_results = {
        'timestamp': datetime.now().strftime("%Y%m%d_%H%M%S"),
        'all_fold_results': all_fold_results,
        'best_hyperparameters': hp_manager.best_params,
        'average_accuracy': float(avg_accuracy),
        'std_accuracy': float(std_accuracy)
    }
    
    with open('final_training_results.json', 'w') as f:
        json.dump(final_results, f, indent=4)

Using 2 GPUs!
Detected classes: {'AD': 0, 'CN': 1, 'MCI': 2}


  AD: 453 örnek
  CN: 748 örnek
  MCI: 981 örnek


Class weights: tensor([1.6056, 0.9724, 0.7414], device='cuda:0')

>>> Fold 1/5


/tmp/ipykernel_19/972150201.py:380: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()


Epoch 1/10


...


Train:  41%|████      | 45/110 [11:44<20:21, 18.80s/it]


Train:  42%|████▏     | 46/110 [11:49<15:37, 14.65s/it]


Train:  43%|████▎     | 47/110 [11:54<12:22, 11.79s/it]


Train:  44%|████▎     | 48/110 [11:59<10:07,  9.80s/it]


Train:  45%|████▍     | 49/110 [12:42<20:00, 19.68s/it]